# Challenge: 

# Imports

In [2]:
import torch;
from torch import nn;
import numpy;
import matplotlib.pyplot as plt;
import pandas as pd;
import sklearn;

from ForwardModelNN import FeedForwardNN;

device: str = "cuda"  if torch.cuda.is_available() else "cpu";

# Installing Dependencies

In [ ]:
%pip install pandas
%pip install numpy

# Data

## Loading the Data

In [ ]:
df = pd.read_csv('dataset/participant_data_cleaned_median.csv');
column_names = df.columns;
for col in column_names: print("Feature: " + col);

df['_time'] = pd.to_datetime(df['_time'])
df

In [ ]:
import numpy as np;
import random;
origData = df.to_numpy();
randSample = origData[random.randint(0,len(origData)-1)]; # for inspection

print("Type of Array with original data: " + str(type(origData)));
print("Type of a record in the original data: " + str(type(randSample)));
print("Number of records in the original dataset: " + str(len(origData)));
print("Number of features in a record: " + str(randSample.size));
print("Dimensionality of a record: " + str(randSample.ndim));
print("Dimensionality of the entire data array: " + str(origData.ndim));
print("Shape of the Array: " + str(origData.shape));
print("Shape of a record: " + str(randSample.shape));

We saw NaN values in the table. That is because of empty cells in the table. To prove our intuition, let's inspect whether there are NaN in specific columns.

In [ ]:
df.isna().any()

## Data Cleansing

We need to replace all the NaN values with 0.

In [ ]:
df = df.fillna(0)
df

In [ ]:
# ensure there is no more NaN values in the table
df.isna().any()

Now there's no more NaN values in table.

In [ ]:
df

In [ ]:
import random;
origData = df.to_numpy();
randSample = origData[random.randint(0,len(origData)-1)]; # for inspection

print("Type of Array with original data: " + str(type(origData)));
print("Type of a record in the original data: " + str(type(randSample)));
print("Number of records in the original dataset: " + str(len(origData)));
print("Number of features in a record: " + str(randSample.size));
print("Dimensionality of a record: " + str(randSample.ndim));
print("Dimensionality of the entire data array: " + str(origData.ndim));
print("Shape of the Array: " + str(origData.shape));
print("Shape of a record: " + str(randSample.shape));

## Visualizing the Data

## Splitting the Datasets 

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# From web
def create_window(target, feature, window=1, offset=0):
    feature_new, target_new = [], []
    feature_np = feature.to_numpy()
    target_np = target.to_numpy()
    for i in range(window, target.shape[0] - offset):
        feature_list = feature_np[i - window:i]
        feature_new.append(feature_list.reshape(window, feature_np.shape[1]))
        target_new.append(target_np[i+offset].reshape(1))
    return np.array(feature_new), np.array(target_new)

# Scale all except date
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.drop(columns='_time'))
df_scaled = pd.DataFrame(df_scaled, columns=df.drop(columns='_time').columns)

window = 30
feature_columns = ["Latitude", "Longitude", "Altitude", 'http_result', 'icmp_result', 'tcp_result', 'udp_result']

feature, target = create_window(df_scaled['http_result'],df_scaled[feature_columns], window=window)

print(feature[0])
print(target[0])
print(df_scaled.head(12))

x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.2)

# Machine Learning Modelling

## Creating the Model

In [ ]:
import tensorflow as tf
from keras import layers


print(x_train.shape)

model = tf.keras.Sequential([
    layers.LSTM(64, input_shape=[x_train.shape[1], x_train.shape[2]]),
    layers.Dense(1)])

model.compile(
    # optimizer=tf.keras.optimizers.RMSprop(0.0001),
    optimizer='RMSProp', 
    loss='mse')


## Fitting the Model with Training Data

In [ ]:
model.summary()

history = model.fit(
    x_train,
    y_train,
    validation_split=0.3,
    epochs=50
)

## Verify if Fitting is performed correctly

# Perform Predictions with the Model

# Evaluate the Performance of the Model

# Conclusion and Further Discussion